In [0]:
import pandas as pd
import numpy as np 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import keras

In [0]:
df= pd.read_csv('toronto_val.csv')
del df['Unnamed: 0']

In [84]:
test_df= pd.read_csv('toronto_test.csv')
del df['Unnamed: 0']

KeyError: ignored

In [0]:
df=df[df.describe().columns]
test_df=test_df[df.describe().columns]

In [0]:
del df['elem']
del test_df['elem']
del df['biz_num']
del test_df['biz_num']

In [89]:
df.head()

,stars_revs,latitude,longitude,review_count,stars_bus,user_stars_mean,user_review_num,svd,svd_pp,nmf,so,ccl,np
0,5.0,43.792427,-79.249860,6,3.703704,2.555556,10,3.259647,3.132973,2.694755,2.200000,3.444736,3.444736
1,1.0,43.654646,-79.422938,85,3.990327,2.933333,16,3.225836,2.984553,3.645331,3.645331,3.645331,3.645331
2,1.0,43.656008,-79.393558,431,3.474337,3.463415,206,3.225836,2.984553,3.645331,3.645331,3.645331,3.645331
3,4.0,43.654321,-79.402084,84,3.011628,3.750000,53,3.225836,2.984553,3.645331,3.645331,3.645331,3.645331
4,5.0,43.664492,-79.377156,118,3.824710,4.750000,5,3.179841,3.321179,2.378970,2.375000,2.767249,2.767249


In [93]:
test_df.head()

,stars_revs,latitude,longitude,review_count,stars_bus,user_stars_mean,user_review_num,svd,svd_pp,nmf,so,ccl,np
0,5.0,43.654932,-79.416348,20,4.093182,3.570223,1,3.645331,3.645331,3.645331,3.645331,3.645331,3.645331
1,4.0,43.662988,-79.331801,48,3.779820,3.833333,7,3.593482,3.641347,3.645331,3.645331,3.645331,3.645331
2,2.0,43.645241,-79.397404,286,3.453695,3.443038,80,3.593482,3.641347,3.645331,3.645331,3.645331,3.645331
3,3.0,43.773543,-79.413796,209,3.639408,3.638298,48,3.232565,3.169770,3.645331,3.645331,3.645331,3.645331
4,4.0,43.653225,-79.397247,344,3.569389,3.428571,8,3.232565,3.169770,3.645331,3.645331,3.645331,3.645331


In [0]:
train=df[:int(0.9*len(df))]
val= df[int(0.9*len(df)):]

In [0]:
x_train= train[train.columns[1:]]
x_val= val[val.columns[1:]]
x_test=test_df[test_df.columns[1:]]
y_train= pd.DataFrame(train[train.columns[0]])
y_val= pd.DataFrame(val[val.columns[0]])
y_test= test_df[test_df.columns[0]]

In [96]:
for col in x_train.columns:
  x_val[col]= (x_val[col]-x_train[col].mean())/(x_train[col].std())
  x_test[col]= (x_test[col]-x_train[col].mean())/(x_train[col].std())
  x_train[col]= (x_train[col]-x_train[col].mean())/(x_train[col].std())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [97]:
np.round(x_train.describe(),5)

,latitude,longitude,review_count,stars_bus,user_stars_mean,user_review_num,svd,svd_pp,nmf,so,ccl,np
count,96129.00000,96129.00000,96129.00000,96129.00000,96129.00000,96129.00000,96129.00000,96129.00000,96129.00000,96129.00000,96129.00000,96129.00000
mean,-0.00000,0.00000,0.00000,0.00000,-0.00000,0.00000,-0.00000,-0.00000,-0.00000,-0.00000,-0.00000,-0.00000
std,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
min,-1.89603,-6.28995,-1.03720,-6.14230,-3.90855,-0.51367,-6.45838,-6.85615,-5.47232,-5.46920,-5.63850,-5.63850
25%,-0.58588,-0.41063,-0.71699,-0.58592,-0.34253,-0.48119,-0.47393,-0.51255,0.10683,0.01197,-0.02222,-0.02222
50%,-0.39934,-0.03109,-0.30945,0.11477,-0.09659,-0.36751,-0.00288,-0.00860,0.10683,0.01197,-0.02222,-0.02222
75%,0.04052,0.23835,0.42995,0.67828,0.51824,0.05474,0.46439,0.52864,0.10683,0.01197,-0.02222,-0.02222
max,4.64891,5.10008,5.65227,2.66627,1.99383,11.11446,3.79480,3.53100,2.96390,2.81887,2.85387,2.85387


In [98]:
np.round(x_val.describe(),5)

,latitude,longitude,review_count,stars_bus,user_stars_mean,user_review_num,svd,svd_pp,nmf,so,ccl,np
count,10682.00000,10682.00000,10682.00000,10682.00000,10682.00000,10682.00000,10682.00000,10682.00000,10682.00000,10682.00000,10682.00000,10682.00000
mean,-0.05794,-0.05433,0.10647,0.07524,-0.00260,0.00805,0.11814,0.09854,-0.09121,0.02229,0.02715,0.02715
std,0.92929,0.86150,1.09571,0.97240,0.98688,1.02705,0.88986,0.87698,1.16217,1.17467,1.18532,1.18532
min,-1.74934,-4.31423,-1.03720,-5.41164,-3.90855,-0.51367,-6.30894,-6.13689,-5.47232,-5.46920,-5.63850,-5.63850
25%,-0.59010,-0.37859,-0.65877,-0.52897,-0.33783,-0.48119,-0.10847,-0.13825,0.10683,0.01197,-0.02222,-0.02222
50%,-0.43050,-0.07622,-0.26287,0.20718,-0.08293,-0.36751,-0.00288,-0.00860,0.10683,0.01197,-0.02222,-0.02222
75%,-0.07041,0.18988,0.47070,0.75016,0.51824,0.07098,0.42459,0.49619,0.10683,0.01197,-0.02222,-0.02222
max,3.72852,5.08873,3.81252,2.62568,1.99383,11.11446,3.79480,3.53100,2.96390,2.81887,2.85387,2.85387


In [99]:
y_train.head()

,stars_revs
0,5.0
1,1.0
2,1.0
3,4.0
4,5.0


In [0]:
act='linear'
act2= 'linear'
reg_rate=0
init_bias= keras.initializers.Constant(value=1)
dropout_rate=0

model = Sequential()
model.add(Dense(12, input_dim=12, kernel_initializer='random_uniform', bias_initializer=init_bias, activation=act))

model.add(Dense(12, activation=act,  bias_initializer=init_bias,activity_regularizer=keras.regularizers.l2(reg_rate)))

model.add(Dense(10, activation=act,  bias_initializer=init_bias,activity_regularizer=keras.regularizers.l2(reg_rate)))

model.add(Dropout(dropout_rate))
#keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)


model.add(Dense(8, activation=act,  bias_initializer=init_bias,activity_regularizer=keras.regularizers.l2(reg_rate)))
model.add(Dense(6, activation=act2,  bias_initializer=init_bias,activity_regularizer=keras.regularizers.l2(reg_rate)))

model.add(Dense(4, activation=act2,  bias_initializer=init_bias,activity_regularizer=keras.regularizers.l2(reg_rate)))

model.add(Dropout(dropout_rate))
#keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)




model.add(Dense(3, activation=act2,  bias_initializer=init_bias,activity_regularizer=keras.regularizers.l2(reg_rate)))



model.add(Dense(1, activation=act2,  bias_initializer=init_bias,activity_regularizer=keras.regularizers.l2(reg_rate)))
model.add(Dense(3, activation=act2,  bias_initializer=init_bias,activity_regularizer=keras.regularizers.l2(reg_rate)))
model.add(Dense(1, activation=act2,  bias_initializer=init_bias,activity_regularizer=keras.regularizers.l2(reg_rate)))

model.compile(loss='mean_squared_error', optimizer='adam')

In [106]:
model.fit(x_train, y_train,
            batch_size=500,
            epochs=50,
            verbose=1,
            validation_data=(x_val, y_val))

Train on 96129 samples, validate on 10682 samples
Epoch 1/500
96129/96129 [==============================] - 2s 25us/step - loss: 2.4907 - val_loss: 1.3056
Epoch 2/500
96129/96129 [==============================] - 1s 5us/step - loss: 1.3311 - val_loss: 1.3011
Epoch 3/500
96129/96129 [==============================] - 1s 5us/step - loss: 1.3297 - val_loss: 1.3016
Epoch 4/500
96129/96129 [==============================] - 1s 6us/step - loss: 1.3307 - val_loss: 1.3064
Epoch 5/500
96129/96129 [==============================] - 1s 5us/step - loss: 1.3312 - val_loss: 1.3016
Epoch 6/500
96129/96129 [==============================] - 1s 5us/step - loss: 1.3298 - val_loss: 1.3042
Epoch 7/500
96129/96129 [==============================] - 1s 6us/step - loss: 1.3298 - val_loss: 1.3011
Epoch 8/500
96129/96129 [==============================] - 1s 6us/step - loss: 1.3308 - val_loss: 1.3024
Epoch 9/500
96129/96129 [==============================] - 1s 6us/step - loss: 1.3307 - val_loss: 1.3006
Epoc

KeyboardInterrupt: ignored

In [0]:
y_v= model.predict(x_val)

In [0]:
y_val['pred']=y_v

In [110]:
y_val.head()

,stars_revs,pred
96129,4.0,4.120045
96130,4.0,4.048193
96131,1.0,2.928698
96132,1.0,3.452840
96133,4.0,4.238001


In [0]:
y_val['pred_bucket']=pd.cut(y_val['pred'], [0,1,1.5,2,2.5,3,3.5,4,4.5,5,6])

In [112]:
y_val.groupby('pred_bucket').mean()

,stars_revs,pred
pred_bucket,,
"(0.0, 1.0]",1.000000,0.954713
"(1.0, 1.5]",1.666667,1.343751
"(1.5, 2.0]",2.000000,1.826625
"(2.0, 2.5]",2.488372,2.305097
"(2.5, 3.0]",2.819149,2.808919
"(3.0, 3.5]",3.227292,3.287916
"(3.5, 4.0]",3.728180,3.761371
"(4.0, 4.5]",4.174289,4.197454
"(4.5, 5.0]",4.573333,4.646935


In [115]:
y_val.groupby('pred_bucket').count()

,stars_revs,pred
pred_bucket,,
"(0.0, 1.0]",2,2
"(1.0, 1.5]",6,6
"(1.5, 2.0]",33,33
"(2.0, 2.5]",172,172
"(2.5, 3.0]",658,658
"(3.0, 3.5]",2345,2345
"(3.5, 4.0]",4308,4308
"(4.0, 4.5]",2777,2777
"(4.5, 5.0]",375,375


In [0]:
y_p=model.predict(x_test)

In [117]:
np.array(y_p).shape

(15259, 1)

In [0]:
y_t=pd.DataFrame(y_test)

In [0]:
y_t['pred']=y_p

In [120]:
y_t.head()

,stars_revs,pred
0,5.0,4.029197
1,4.0,3.857118
2,2.0,3.435829
3,3.0,3.662510
4,4.0,3.532968


In [0]:
y_t['pred_bucket']=pd.cut(y_t.pred,[0,1,2,3,4,5,6])

In [122]:
y_t.head()

,stars_revs,pred,pred_bucket
0,5.0,4.029197,"(4, 5]"
1,4.0,3.857118,"(3, 4]"
2,2.0,3.435829,"(3, 4]"
3,3.0,3.662510,"(3, 4]"
4,4.0,3.532968,"(3, 4]"


In [123]:
((sum((y_t.stars_revs - y_t.pred)**2))/len(y_t))**0.5

1.137098723302045

In [124]:
y_t.groupby('pred_bucket').mean()

,stars_revs,pred
pred_bucket,,
"(0, 1]",3.000000,0.507236
"(1, 2]",1.827586,1.721724
"(2, 3]",2.682152,2.721151
"(3, 4]",3.541453,3.575030
"(4, 5]",4.260662,4.248562
"(5, 6]",4.766667,5.108453


In [125]:
y_t.groupby('pred_bucket').count()

,stars_revs,pred
pred_bucket,,
"(0, 1]",1,1
"(1, 2]",58,58
"(2, 3]",1227,1227
"(3, 4]",9746,9746
"(4, 5]",4197,4197
"(5, 6]",30,30
